In [1]:
import logging                                                                                                                                               
import matplotlib.pylab as plt                                                                                                                                                  
import numpy as np                                                                                                                                                              
import pandas as pd                                                                                                                                                            
import torch                                                                                                                                                                 
import torch.nn as nn                                                                                                                                                      
import torch.optim as optim                                                                                                                                              
import torchvision.transforms as transforms                                                                                                                        
import sys                                                                                                                                                              
                                                                                                                                                                         
from argparse import ArgumentParser                                                                                                                                          
from datetime import datetime                                                                                                                                          
from pathlib import Path                                                                                                                                              
from PIL import Image                                                                                                                                                     
from sklearn import preprocessing                                                                                                                                          
from sklearn.metrics import precision_recall_fscore_support                                                                                                               
from sklearn.model_selection import train_test_split                                                                                                                        
from torch.optim.lr_scheduler import StepLR                                                                                                                                 
from torchvision import datasets, transforms                                                                                                                                 
from torch.utils.data import Dataset, DataLoader                                                                                                                              
from types import SimpleNamespace  

import copy

In [2]:
im_path = '/home/hnaik/data/qualia/2d_shapes/64x64/circles/green/r18-c38_d34_t3.png'

In [3]:
trans = transforms.Compose([
    transforms.ToTensor()
])

criterion = nn.BCELoss()
im = Image.open(im_path)
X = trans(im)
X = X.unsqueeze(0)

In [4]:
net = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3),
    nn.Flatten(),
    nn.Linear(3136, 4),
    nn.Sigmoid()
)

label = torch.tensor([0., 1., 0., 0.])
output = net(X)

loss = criterion(output, label)

/home/hnaik/.virtualenvs/iconn-yLpHbsxh/lib/python3.8/site-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([1, 4])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


In [5]:
net

Sequential(
  (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten()
  (7): Linear(in_features=3136, out_features=4, bias=True)
  (8): Sigmoid()
)

In [6]:
loss.backward()

In [7]:
net

Sequential(
  (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten()
  (7): Linear(in_features=3136, out_features=4, bias=True)
  (8): Sigmoid()
)

In [11]:
class IpAutograd(torch.autograd.Function):
    @staticmethod
    def forward(ctx, X):
        return X
    
    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()
        print(grad_input.shape)
        return grad_input
    

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.stage_1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, padding=2),
            nn.ReLU(inplace=True)
        )
        
        self.max_pool = nn.MaxPool2d(kernel_size=3)
        
        self.stage_2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)    
        )
        
        self.ag = IpAutograd()
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3136, 4),
            nn.Sigmoid()
        )
    def forward(self, X):
        X = self.max_pool(IpAutograd.apply((self.stage_1(X))))
        X = self.max_pool(IpAutograd.apply(self.stage_2(X)))
        return self.classifier(X)
    
    def backward(self, grad_output):
        print(grad_output)
        
        return grad_output

inet = Net()
output = inet(X)

output.shape

torch.Size([1, 4])

In [12]:
loss = criterion(output, label)
loss.backward()

torch.Size([1, 64, 21, 21])
torch.Size([1, 32, 64, 64])


In [ ]:
# for im in output[0]:
#     plt.imshow(im.detach().numpy(), cmap='gray')
#     plt.show()

In [ ]:
for m in inet:
    print(m.training)

In [ ]:
# z = np.random.rand(64 * 64).reshape([64, 64])
x = copy.deepcopy(output[0][0].detach().numpy())
# z = output[0][0].detach().numpy()
tau = 5
beta = 0.25
am = np.argmax(x)
n = len(x[0])
u_i = am // n
u_j = am % n
print(am, n, u_i, u_j)
u = (u_i, u_j)

def norm_1(x1, x2):
    return abs(x1[0] - x2[0]) + abs(x1[1] - x2[1])

import math
def norm_2(x1, x2):
    return np.sqrt(math.pow((x1[0] - x2[0]), 2) + math.pow((x1[1] - x2[1]), 2))

u = (13, 7)

In [ ]:
T_1 = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        T_1[i, j] = tau * np.max(1 - (beta * (norm_2((i, j), u) / n)), -1)
  
T_1.fill(-tau)
plt.imshow(T_1, cmap='gray')
plt.show()

In [ ]:
plt.imshow(x, cmap='gray')
plt.show()

In [ ]:
xx = x * T_1

plt.imshow(xx, cmap='gray')
plt.show()

In [ ]:
for x in inet.parameters():
    print(x.shape)
    
output.shape